This part is responsible for getting stock market data, it is done by crawling the website yahoo finance and getting the data for our desired time frame. The chosen resources are Coal, Brent Oil and Natural Gas.

In [11]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime


# gets data from yahoo finance with the given url, the filename and resource have to be put in
def get_Data(url, filename, resource):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')

    # Extract headers
    headers = [header.text.strip() for header in table.find_all('th')]

    # Close column = Value for end of the day, the rest of the columns are not needed
    close_column_index = next(i for i, header in enumerate(headers) if header.lower().startswith('close'))

    # Extract data from the table
    data = []
    for row in table.find_all('tr'):
        columns = row.find_all('td')
        if columns:
            # Get correcct Date format
            date = columns[0].text.strip()
            date = datetime.strptime(date, '%b %d, %Y').strftime('%Y-%m-%d')
            close_value = columns[close_column_index].text.strip()
            data.append([date, close_value])


    # data is in the wrong order, put it from earliest to latest
    data.sort(key=lambda x: datetime.strptime(x[0], '%Y-%m-%d'))

    # Save the data with headers as a CSV file
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Date', resource])
        writer.writerows(data)

    print("Data saved as ",filename)

In [12]:
url_brent = 'https://finance.yahoo.com/quote/BZ%3DF/history/?period1=1420070400&period2=1734903292&guccounter=1'
url_gas = 'https://finance.yahoo.com/quote/NG%3DF/history/?period1=1420070400&period2=1734905333'
url_coal = 'https://finance.yahoo.com/quote/MTFZ24.NYM/history/?period1=1420070400&period2=1734905597'

get_Data(url_brent, '../data_collection/brent_oil.csv', 'Brent Oil')
get_Data(url_gas, '../data_collection/naturalGas.csv', 'Natural Gas')
get_Data(url_coal, '../data_collection/coal.csv', 'Coal')

Data saved as  ../data_collection/brent_oil.csv
Data saved as  ../data_collection/naturalGas.csv
Data saved as  ../data_collection/coal.csv


In [13]:
import pandas as pd

#the data is missing hour, as it is only daily, fills weekend gaps also
def fill_missing_hours(csv):
    df = pd.read_csv(csv)

    value_Name = df.columns[1]

    # Manually parse the 'date' column using the correct format (DD.MM.YY)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df['Date'] = df['Date'].dt.normalize()
    df.set_index('Date', inplace=True)

    # start 2015
    full_hourly_range = pd.date_range(start='01.01.2015', end=df.index.max() + pd.Timedelta(days=1), freq='H')[:-1]

    # put prefered null value here
    df_full = df.reindex(full_hourly_range, fill_value=pd.NA)
    df_full.reset_index(inplace=True)
    df_full.rename(columns={'index': 'Date'}, inplace=True)
    df_full[value_Name] = df_full.groupby(df_full['Date'].dt.floor('D'))[value_Name].transform(lambda group: group.ffill().bfill())

    # fills emptys
    df_full[value_Name].fillna('', inplace=True)
    df_full.to_csv(csv, index=False)
    print('Missing Hours Filled: ', csv)


fill_missing_hours('../data_collection/brent_oil.csv')
fill_missing_hours('../data_collection/naturalGas.csv')
fill_missing_hours('../data_collection/coal.csv')

df1 = pd.read_csv('../data_collection/brent_oil.csv')
df2 = pd.read_csv('../data_collection/naturalGas.csv')
df3 = pd.read_csv('../data_collection/coal.csv')

merged_df = pd.merge(df1, df2, on='Date', how='outer')
merged_df = pd.merge(merged_df, df3, on='Date', how='outer')

merged_df.to_csv('../data_collection/merged_data.csv', index=False)

print("CSV files have been merged and saved.")


/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/3023880499.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_hourly_range = pd.date_range(start='01.01.2015', end=df.index.max() + pd.Timedelta(days=1), freq='H')[:-1]
/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/3023880499.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full[value_Name].fillna('', inplace=True)
/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_1489

Missing Hours Filled:  ../data_collection/brent_oil.csv


/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/3023880499.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full[value_Name].fillna('', inplace=True)
/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/3023880499.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_full[value_Name].fillna('', inplace=True)
/var/fol

Missing Hours Filled:  ../data_collection/naturalGas.csv


/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/3023880499.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full[value_Name].fillna('', inplace=True)
/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/3023880499.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_full[value_Name].fillna('', inplace=True)


Missing Hours Filled:  ../data_collection/coal.csv
CSV files have been merged and saved.


This part is responsible for getting the ENTSO-DATA.

The ENTSO-E (European Network of Transmission System Operators for Electricity) Transparency Platform, launched in 2015, serves as Europe's central hub for electricity market data. Created to fulfill EU Regulation 543/2013, it collects and publishes data from over 42 transmission system operators across Europe. The platform transformed what was once a fragmented landscape of national data sources into a unified repository, providing crucial information about power generation, consumption, and cross-border flows. For market participants and analysts, this data source is invaluable as it offers insights into the fundamental drivers of electricity prices, including generation mix, grid constraints, and demand patterns.

![Example Image](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRv8tncnUyen1EF-uNSqu9D2iw7FSClbohI4KQBGH2XPhI_7ve-W9BXXI5-vluG2t9FCLQ&usqp=CAU)

We applied for an API-Key and programmatically extract relevant information in the following.

We extracted day-ahead prices, generation mix, and cross-border flows for the year 2020. We also queried some predictions, including the generation mix for the next week, which might hint to the future day-ahead prices.

In [14]:
# fetching data with right naming conventions

from entsoe import EntsoeRawClient, EntsoePandasClient
import pandas as pd
import numpy as np
from dotenv import load_dotenv 
import os
import datetime
import time
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

load_dotenv()
ENTSOE_API_KEY="562a20c4-03b0-4ee6-a692-19d534b4393a"
client = EntsoePandasClient(api_key=ENTSOE_API_KEY)

start = pd.Timestamp('20150101', tz='UTC')
change_date = pd.Timestamp('20181001', tz='UTC')
end = pd.Timestamp(datetime.datetime.now(), tz='UTC')

print(os.getcwd())
out_dir = '../data_collection'

country_code_old = 'DE_AT_LU'
country_code_new = 'DE_LU'

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10), retry=retry_if_exception_type(Exception))
def query_entsoe_data(query_func, country_code, start, end):
    try:
        df = query_func(country_code, start=start, end=end)
        
    except Exception as e:
        print(f"Error querying data: {e}")
        raise
    return df

def merge_data(query_func):
    data_old = query_entsoe_data(query_func, country_code_old, start, change_date)
    
    data_new = query_entsoe_data(query_func, country_code_new, change_date, end)

    if not isinstance(data_old, pd.DataFrame):
        data_old = data_old.to_frame()
    if not isinstance(data_new, pd.DataFrame):
        data_new = data_new.to_frame()
    
    if not data_old.empty and not data_new.empty:
        if len(data_old.columns) != len(data_new.columns):
            same_columns = list(set(data_old.columns) & set(data_new.columns))
            data_old = data_old[same_columns]
            data_new = data_new[same_columns]
        else:
            data_new.columns = data_old.columns
    df_combined = pd.concat([data_old, data_new])
    df_combined.index = df_combined.index.tz_convert('UTC')
    return df_combined

def save_df_with_timestamp(df, filename):
    df_copy = df.copy()
    df_copy.index.name = 'Date'
    df_copy.to_csv(filename)

# Day-ahead prices (EUR/MWh)
day_ahead_prices = merge_data(client.query_day_ahead_prices)
day_ahead_prices = day_ahead_prices.rename(columns={day_ahead_prices.columns[0]: 'day_ahead_prices_EURO'})
save_df_with_timestamp(day_ahead_prices, '../data_collection/day_ahead_prices.csv')
print('Day-ahead prices done')

# Load forecast (MWh)
load_forecast = merge_data(client.query_load_forecast)
load_forecast = load_forecast.rename(columns={load_forecast.columns[0]: 'E_load_forecast_MWh'})
save_df_with_timestamp(load_forecast, '../data_collection/load_forecast.csv')
print('Load forecast done')

# Generation forecast (MWh)
generation_forecast = merge_data(client.query_generation_forecast)
generation_forecast = generation_forecast.rename(columns={generation_forecast.columns[0]: 'E_generation_forecast_MWh'})
save_df_with_timestamp(generation_forecast, '../data_collection/generation_forecast.csv')
print('Generation forecast done')

# Wind and solar forecasts (MWh)
intraday_wind_solar_forecast = merge_data(client.query_intraday_wind_and_solar_forecast)
for col in intraday_wind_solar_forecast.columns:
    if 'Wind' in col:
        intraday_wind_solar_forecast = intraday_wind_solar_forecast.rename(columns={col: 'E_wind_forecast_MWh'})
    elif 'Solar' in col:
        intraday_wind_solar_forecast = intraday_wind_solar_forecast.rename(columns={col: 'E_solar_forecast_MWh'})
save_df_with_timestamp(intraday_wind_solar_forecast, '../data_collection/intraday_wind_solar_forecast.csv')
print('Intraday wind and solar forecast done')

# Day ahead wind and solar forecast (MWh)
day_ahead_wind_solar_forecast = merge_data(client.query_wind_and_solar_forecast)
for col in day_ahead_wind_solar_forecast.columns:
    if 'Wind' in col:
        day_ahead_wind_solar_forecast = day_ahead_wind_solar_forecast.rename(columns={col: 'E_wind_forecast_MWh'})
    elif 'Solar' in col:
        day_ahead_wind_solar_forecast = day_ahead_wind_solar_forecast.rename(columns={col: 'E_solar_forecast_MWh'})
save_df_with_timestamp(day_ahead_wind_solar_forecast, '../data_collection/day_ahead_wind_solar_forecast.csv')
print('Day ahead wind and solar forecast done')

# Physical crossborder flows (MWh)
physical_crossborder_flows = merge_data(lambda cc, start, end: client.query_physical_crossborder_allborders(start=start, end=end, country_code=cc, export=True))
for col in physical_crossborder_flows.columns:
    physical_crossborder_flows = physical_crossborder_flows.rename(columns={col: f'E_crossborder_{col}_actual_MWh'})
save_df_with_timestamp(physical_crossborder_flows, '../data_collection/physical_crossborder_flows.csv')
print('Physical crossborder flows done')


/Users/clarahuefner/Desktop/final-submission-main/merged_data/scripts
Day-ahead prices done
Load forecast done
Generation forecast done
Intraday wind and solar forecast done
Day ahead wind and solar forecast done
Physical crossborder flows done


In [20]:
df4 = pd.read_csv('../data_collection/day_ahead_prices.csv')
df5 = pd.read_csv('../data_collection/load_forecast.csv')
df6 = pd.read_csv('../data_collection/generation_forecast.csv')
df7 = pd.read_csv('../data_collection/intraday_wind_solar_forecast.csv')
df8 = pd.read_csv('../data_collection/day_ahead_wind_solar_forecast.csv')
df9 = pd.read_csv('../data_collection/physical_crossborder_flows.csv')


merged_df2 = pd.merge(df5, df4, on='Date', how='outer')
merged_df2 = pd.merge(merged_df2, df6, on='Date', how='outer')
merged_df2 = pd.merge(merged_df2, df7, on='Date', how='outer')
merged_df2 = pd.merge(merged_df2, df8, on='Date', how='outer')
merged_df2 = pd.merge(merged_df2, df9, on='Date', how='outer')

merged_df2.to_csv('../data_collection/merged_data2.csv', index=False)

df = pd.read_csv('../data_collection/merged_data2.csv')

df['Date'] = pd.to_datetime(df['Date'])
df_filtered = df[df['Date'].dt.minute == 0]
df_filtered['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_filtered.to_csv('../data_collection/merged_data3.csv', index=False)

/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/922687961.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d %H:%M:%S')


This is responsible for the Covid Lockdown Data.

In [16]:
FILE_URL = 'https://pada.psycharchives.org/bitstream/9ff033a9-4084-4d0e-87eb-aa963a1324a5'
covid_df = pd.read_csv(FILE_URL, sep=",", header=[0])
print(covid_df.head().iloc[:,:5])

# dict with influence of measure (see readme)
measure_influence = {
    'leavehome': 1,
    'dist': 0,
    'msk': 1,
    'shppng': 2,
    'hcut': 2,
    'ess_shps': 2,
    'zoo': 0,
    'demo': 0,
    'school': 1,
    'church': 0,
    'onefriend': 0,
    'morefriends': 0,
    'plygrnd': 0,
    'daycare': 2,
    'trvl': 1,
    'gastr': 2
}
# dict with state relative population of country
state_percentages = {
    'Baden-Wuerttemberg': 0.133924061,
    'Bayern': 0.158676851,
    'Berlin': 0.044670274,
    'Brandenburg': 0.030491172,
    'Bremen': 0.008169464,
    'Hamburg': 0.022560236,
    'Hessen': 0.075833,
    'Mecklenburg-Vorpommern': 0.019245033,
    'Niedersachsen': 0.096398323,
    'Nordrhein-Westfalen': 0.214840756,
    'Rheinland-Pfalz': 0.049301337,
    'Saarland': 0.011744796,
    'Sachsen': 0.048299274,
    'Sachsen-Anhalt': 0.025752514,
    'Schleswig-Holstein': 0.035026746,
    'Thueringen': 0.025066162
}

def evaluate_date(request_date):
    if request_date in list(covid_df):
        truncated_covid_df = covid_df[['state', 'Measure ', request_date]]
        sum_value = 0
        for index, row in truncated_covid_df.iterrows():
            if row.isnull().values.any(): continue  # if any value in row is missing
            if measure_influence[row['Measure ']] == 0: continue  # if measure has no influence
            sum_value += ((int(row[request_date]) / 5) + 0.6) * state_percentages[row['state']] * measure_influence[
                row['Measure ']]  # see readme documentation
        return sum_value
    else:
        return 0
    
# generate and populate dataframe with all dates from 2015-1-1 - today
from datetime import date, timedelta

working_dt = date(2015, 1, 1)
end_dt = date(date.today().year, date.today().month, date.today().day)
delta = timedelta(days=1)

data_rows = []

# populate df
while working_dt <= end_dt:
    factor = evaluate_date(working_dt.isoformat())
    date = working_dt.isoformat()
    for hour in range(24):
        timestamp = pd.Timestamp(working_dt.isoformat()) + pd.Timedelta(hours=hour)
        data_rows.append({'Date': timestamp, 'Covid factor': factor})  # Add to rows list
    working_dt += delta

covid_factors_df = pd.DataFrame(data_rows)
print(covid_factors_df.head)

covid_factors_df.to_csv('../data_collection/covid.csv', index=False)

   Unnamed: 0               state   Measure   2020-03-08  2020-03-09
0         1.0  Baden-Wuerttemberg  leavehome         0.0         0.0
1         1.0  Baden-Wuerttemberg       dist         0.0         0.0
2         1.0  Baden-Wuerttemberg        msk         0.0         0.0
3         1.0  Baden-Wuerttemberg     shppng         0.0         0.0
4         1.0  Baden-Wuerttemberg       hcut         0.0         0.0
<bound method NDFrame.head of                      Date  Covid factor
0     2015-01-01 00:00:00           0.0
1     2015-01-01 01:00:00           0.0
2     2015-01-01 02:00:00           0.0
3     2015-01-01 03:00:00           0.0
4     2015-01-01 04:00:00           0.0
...                   ...           ...
87475 2024-12-23 19:00:00           0.0
87476 2024-12-23 20:00:00           0.0
87477 2024-12-23 21:00:00           0.0
87478 2024-12-23 22:00:00           0.0
87479 2024-12-23 23:00:00           0.0

[87480 rows x 2 columns]>


This is responsible for the SMARD Data.

In [17]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import requests, datetime
from io import StringIO
#from datetime import datetime

#-------------translation for Balancing:------------------
balancing_id={
    #automatic frequency, tag=af
    "automatic_frequency":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Volume activated (+) [MWh] Calculated resolutions":"af_E_Volume_Activated_Plus_MWh",
        "Volume activated (-) [MWh] Calculated resolutions":"af_E_Volume_Activated_Minus_MWh",
        "Activation price (+) [€/MWh] Calculated resolutions":"af_Activation_Price_Plus_EUR/MWh",
        "Activation price (-) [€/MWh] Calculated resolutions":"af_Activation_Price_Minus_EUR/MWh",
        "Volume procured (+) [MW] Calculated resolutions":"af_E_Volume_Procured_Plus_MW",
        "Volume procured (-) [MW] Calculated resolutions":"af_E_Volume_Procured_Minus_MW",
        "Procurement price (+) [€/MW] Calculated resolutions":"af_Procurement_Price_Plus_EUR/MW",
        "Procurement price (-) [€/MW] Calculated resolutions":"af_Procurement_Price_Minus_EUR/MW",
    },
    #tag=mf
    "manual_frequency":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Volume activated (+) [MWh] Calculated resolutions":"mf_E_Volume_Activated_Plus_MWh",
        "Volume activated (-) [MWh] Calculated resolutions":"mf_E_Volume_Activated_Minus_MWh",
        "Activation price (+) [€/MWh] Calculated resolutions":"mf_Activation_Price_Plus_EUR/MWh",
        "Activation price (-) [€/MWh] Calculated resolutions":"mf_Activation_Price_Minus_EUR/MWh",
        "Volume procured (+) [MW] Calculated resolutions":"mf_E_Volume_Procured_Plus_MW",
        "Volume procured (-) [MW] Calculated resolutions":"mf_E_Volume_Procured_Minus_MW",
        "Procurement price (+) [€/MW] Calculated resolutions":"mf_Procurement_Price_Plus_EUR/MW",
        "Procurement price (-) [€/MW] Calculated resolutions":"mf_Procurement_Price_Minus_EUR/MW",
    },
     #balancing energy
    "balancing_energy":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Volume (+) [MWh] Calculated resolutions":"E_Volume_Calculated_Plus_MWh",
        "Volume (-) [MWh] Calculated resolutions":"E_Volume_Calculated_Minus_MWh",
        "Price [€/MWh] Calculated resolutions":"Price_Calculated_EUR/MWh",
        "Net income [€] Calculated resolutions":"Net_Income_EUR",
    },
    #costs
    "costs":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Balancing services [€] Calculated resolutions":"Balancing_Services_Calculated_EUR",
        "Network security [€] Calculated resolutions":"Network_Security_Calculated_EUR",
        "Countertrading [€] Calculated resolutions":"Countertrading_Calculated_EUR",
    },
    #frequency_containment_reserve
    "frequency_containment":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Volume procured [MW] Calculated resolutions":"E_Volume_Procured_Calculated_MW",
        "Procurement price [€/MW] Calculated resolutions":"Price_Procument_Calculated_EUR/MW"
    },
    "imported_balancing_services":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Austria [MWh] Calculated resolutions":"import_E_Austria_Calculated_MWh",
    },
    "exported_balancing_services":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Austria [MWh] Calculated resolutions":"export_E_Austria_Calculated_MWh",
    }         
}    

#actual consumption tag=actual
electricity_consumption_id={
    "actual":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Total (grid load) [MWh] Calculated resolutions":"actual_E_Total_Gridload_MWh",
        "Residual load [MWh] Calculated resolutions":"actual_E_Residual_Load_MWh",
        "Hydro pumped storage [MWh] Calculated resolutions":"actual_E_Hydro_Pumped_Storage_MWh",
    },
    #forecasted consumption tag=forecast
    "forecast":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Total (grid load) [MWh] Calculated resolutions":"forecast_E_Total_Gridload_MWh",
        "Residual load [MWh] Calculated resolutions":"forecast_actual_E_Residual_Load_MWh"
    }
}

electricity_generation_id={
    #actual generation
    "actual":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Biomass [MWh] Calculated resolutions":"actual_generation_E_Biomass_MWh",
        "Hydropower [MWh] Calculated resolutions":"actual_generation_E_Hydropower_MWh",
        "Wind offshore [MWh] Calculated resolutions":"actual_generation_E_Windoffshore_MWh",
        "Wind onshore [MWh] Calculated resolutions":"actual_generation_E_Windonshore_MWh",
        "Photovoltaics [MWh] Calculated resolutions":"actual_generation_E_Photovoltaics_MWh",
        "Other renewable [MWh] Calculated resolutions":"actual_generation_E_OtherRenewable_MWh",
        "Nuclear [MWh] Calculated resolutions":"actual_generation_E_Nuclear_MWh",
        "Lignite [MWh] Calculated resolutions":"actual_generation_E_Lignite_MWh",
        "Hard coal [MWh] Calculated resolutions":"actual_generation_E_HardCoal_MWh",
        "Fossil gas [MWh] Calculated resolutions":"actual_generation_E_FossilGas_MWh",
        "Hydro pumped storage [MWh] Calculated resolutions":"actual_generation_E_HydroPumpedStorage_MWh",
        "Other conventional [MWh] Calculated resolutions":"actual_generation_E_OtherConventional_MWh"
    },
    
    #forecastet generation day ahead
    "forecast":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Biomass [MWh] Calculated resolutions":"forecast_generation_E_Biomass_MWh",
        "Hydropower [MWh] Calculated resolutions":"forecast_generation_E_Hydropower_MWh",
        "Wind offshore [MWh] Calculated resolutions":"forecast_generation_E_Windoffshore_MWh",
        "Wind onshore [MWh] Calculated resolutions":"forecast_generation_E_Windonshore_MWh",
        "Photovoltaics [MWh] Calculated resolutions":"forecast_generation_E_Photovoltaics_MWh",
        "Other renewable [MWh] Calculated resolutions":"forecast_generation_E_OtherRenewable_MWh",
        "Nuclear [MWh] Calculated resolutions":"forecast_generation_E_Nuclear_MWh",
        "Lignite [MWh] Calculated resolutions":"forecast_generation_E_Lignite_MWh",
        "Hard coal [MWh] Calculated resolutions":"forecast_generation_E_HardCoal_MWh",
        "Fossil gas [MWh] Calculated resolutions":"forecast_generation_E_FossilGas_MWh",
        "Hydro pumped storage [MWh] Calculated resolutions":"forecast_generation_E_HydroPumpedStorage_MWh",
        "Other [MWh] Calculated resolutions":"forecast_generation_E_Other_MWh",
        "Total [MWh] Original resolutions":"forecast_generation_E_Total_MWh",
        "Photovoltaics and wind [MWh] Calculated resolutions":"forecast_generation_E_PhotovoltaicsAndWind_MWh",
        "Other [MWh] Original resolutions":"forecast_generation_E_Original_MWh"
    },

    #installed generation capacity
    #key=instGenCapacity
    "installed_generation_capacity":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Biomass [MW] Calculated resolutions":"instGenCapacity_E_Biomass_MW",
        "Hydropower [MW] Calculated resolutions":"instGenCapacity_E_Hydropower_MW",
        "Wind offshore [MW] Calculated resolutions":"instGenCapacity_E_Windoffshore_MW",
        "Wind onshore [MW] Calculated resolutions":"instGenCapacity_E_Windonshore_MW",
        "Photovoltaics [MW] Calculated resolutions":"instGenCapacity_E_Photovoltaics_MW",
        "Other renewable [MW] Calculated resolutions":"instGenCapacity_E_OtherRenewable_MW",
        "Nuclear [MW] Calculated resolutions":"instGenCapacity_E_Nuclear_MW",
        "Lignite [MW] Calculated resolutions":"instGenCapacity_E_Lignite_MW",
        "Hard coal [MW] Calculated resolutions":"instGenCapacity_E_HardCoal_MW",
        "Fossil gas [MW] Calculated resolutions":"instGenCapacity_E_FossilGas_MW",
        "Hydro pumped storage [MW] Calculated resolutions":"instGenCapacity_E_HydroPumpedStorage_MW",
        "Other conventional [MW] Calculated resolutions":"instGenCapacity_E_OtherConventional_MW"
    }
}

market_id={
    #key=dayAhead
    "day_ahead_prices":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Germany/Luxembourg [€/MWh] Original resolutions":"dayAhead_Price_GermanyAndLuxembourg_EUR/MWh",
        "∅ DE/LU neighbours [€/MWh] Original resolutions":"dayAhead_Price_GermanyAndLuxembourgAverage_EUR/MWh",
        "Belgium [€/MWh] Original resolutions":"dayAhead_Price_Belgium_EUR/MWh",
        "Denmark 1 [€/MWh] Original resolutions":"dayAhead_Price_Denmark1_EUR/MWh",
        "Denmark 2 [€/MWh] Original resolutions":"dayAhead_Price_Denmark2_EUR/MWh",
        "France [€/MWh] Original resolutions":"dayAhead_Price_France_EUR/MWh",
        "Netherlands [€/MWh] Original resolutions":"dayAhead_Price_Netherlands_EUR/MWh",
        "Norway 2 [€/MWh] Original resolutions":"dayAhead_Price_Norway2_EUR/MWh",
        "Austria [€/MWh] Original resolutions":"dayAhead_Price_Austria_EUR/MWh",
        "Poland [€/MWh] Original resolutions":"dayAhead_Price_Poland_EUR/MWh",
        "Sweden 4 [€/MWh] Original resolutions":"dayAhead_Price_Sweden4_EUR/MWh",
        "Switzerland [€/MWh] Original resolutions":"dayAhead_Price_Switzerland_EUR/MWh",
        "Czech Republic [€/MWh] Original resolutions":"dayAhead_Price_CzechRepublic_EUR/MWh",
        "DE/AT/LU [€/MWh] Original resolutions":"dayAhead_Price_DE/AT/LU_EUR/MWh",
        "Northern Italy [€/MWh] Original resolutions":"dayAhead_Price_NothernItaly_EUR/MWh",
        "Slovenia [€/MWh] Original resolutions":"dayAhead_Price_Slovenia_EUR/MWh",
        "Hungary [€/MWh] Original resolutions":"dayAhead_Price_Hungary_EUR/MWh"
    },
    
    "cross_border_physical":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Net export [MWh] Calculated resolutions":"E_NetExport_crossBorderPhysical_MWh",
        "Netherlands (export) [MWh] Calculated resolutions":"E_NetherlandExport_corssBorderPhysical_MWh",
        "Netherlands (import) [MWh] Calculated resolutions":"E_NetherlandImport_corssBorderPhysical_MW",
        "Switzerland (export) [MWh] Calculated resolutions":"E_SwitzerlandExport_corssBorderPhysical_MWh",
        "Switzerland (import) [MWh] Calculated resolutions":"E_SwitzerlandImport_corssBorderPhysical_MWh",
        "Denmark (export) [MWh] Calculated resolutions":"E_DenmarkExport_corssBorderPhysical_MWh",
        "Denmark (import) [MWh] Calculated resolutions":"E_Denmark_Import_corssBorderPhysical_MWh",
        "Czech Republic (export) [MWh] Calculated resolutions":"E_CzechrepublicExport_corssBorderPhysical_MWh",
        "Czech Republic (import) [MWh] Calculated resolutions":"E_CzechrepublicImport_corssBorderPhysical_MWh",
        "Luxembourg (export) [MWh] Calculated resolutions":"E_LuxembourgExport_corssBorderPhysical_MWh",
        "Luxembourg (import) [MWh] Calculated resolutions":"E_LuxembourgImport_corssBorderPhysical_MWh",
        "Sweden (export) [MWh] Calculated resolutions":"E_SwedenExport_corssBorderPhysical_MWh",
        "Sweden (import) [MWh] Calculated resolutions":"E_SwedenImportv_corssBorderPhysical_MWh",
        "Austria (export) [MWh] Calculated resolutions":"E_AustriaExport_corssBorderPhysical_MWh",
        "Austria (import) [MWh] Calculated resolutions":"E_AustriaImport_corssBorderPhysical_MWh",
        "France (export) [MWh] Calculated resolutions":"E_FranceExport_corssBorderPhysical_MWh",        
        "France (import) [MWh] Calculated resolutions":"E_FranceImport_corssBorderPhysical_MWh",
        "Poland (export) [MWh] Calculated resolutions":"E_PolandExport_corssBorderPhysical_MWh",
        "Poland (import) [MWh] Calculated resolutions":"E_PolandImport_corssBorderPhysical_MWh",
        "Norway (export) [MWh] Calculated resolutions":"E_NorwayExport_corssBorderPhysical_MWh",
        "Norway (import) [MWh] Calculated resolutions":"E_NorwayImport_corssBorderPhysical_MWh",
        "Belgium (export) [MWh] Calculated resolutions":"E_BelgiumExport_corssBorderPhysical_MWh",
        "Belgium (import) [MWh] Calculated resolutions":"E_BelgiumImport_corssBorderPhysical_MWh",
    },
    "scheudled_commercial_exchanges":{
        "Start date":"Start_Date",
        "End date":"End_Date",
        "Net export [MWh] Calculated resolutions":"E_NetExport_MWh",
        "Netherlands (export) [MWh] Calculated resolutions":"E_NetherlandExport_MWh",
        "Netherlands (import) [MWh] Calculated resolutions":"E_NetherlandImport_MW",
        "Switzerland (export) [MWh] Calculated resolutions":"E_SwitzerlandExport_MWh",
        "Switzerland (import) [MWh] Calculated resolutions":"E_SwitzerlandImport_MWh",
        "Denmark (export) [MWh] Calculated resolutions":"E_DenmarkExport_MWh",
        "Denmark (import) [MWh] Calculated resolutions":"E_Denmark_Import_MWh",
        "Czech Republic (export) [MWh] Calculated resolutions":"E_CzechrepublicExport_MWh",
        "Czech Republic (import) [MWh] Calculated resolutions":"E_CzechrepublicImport_MWh",
        "Luxembourg (export) [MWh] Calculated resolutions":"E_LuxembourgExport_MWh",
        "Luxembourg (import) [MWh] Calculated resolutions":"E_LuxembourgImport_MWh",
        "Sweden (export) [MWh] Calculated resolutions":"E_SwedenExport_MWh",
        "Sweden (import) [MWh] Calculated resolutions":"E_SwedenImport_MWh",
        "Austria (export) [MWh] Calculated resolutions":"E_AustriaExport_MWh",
        "Austria (import) [MWh] Calculated resolutions":"E_AustriaImport_MWh",
        "France (export) [MWh] Calculated resolutions":"E_FranceExport_MWh",        
        "France (import) [MWh] Calculated resolutions":"E_FranceImport_MWh",
        "Poland (export) [MWh] Calculated resolutions":"E_PolandExport_MWh",
        "Poland (import) [MWh] Calculated resolutions":"E_PolandImport_MWh",
        "Norway (export) [MWh] Calculated resolutions":"E_NorwayExport_MWh",
        "Norway (import) [MWh] Calculated resolutions":"E_NorwayImport_MWh",
        "Belgium (export) [MWh] Calculated resolutions":"E_BelgiumExport_MWh",
        "Belgium (import) [MWh] Calculated resolutions":"E_BelgiumImport_MWh",
    }
}

def main():

    output_path = sys.argv[1]

    dict_ids = [balancing_id["automatic_frequency"],
                balancing_id["balancing_energy"],
                balancing_id["costs"],
                balancing_id["exported_balancing_services"],
                balancing_id["frequency_containment"],
                balancing_id["imported_balancing_services"],
                balancing_id["manual_frequency"],
                electricity_consumption_id["actual"],
                electricity_consumption_id["forecast"],
                electricity_generation_id["actual"],
                electricity_generation_id["forecast"],
                market_id["cross_border_physical"],
                market_id["scheudled_commercial_exchanges"],
                market_id["day_ahead_prices"]    
    ]
    
    final_df = None

    for i in range(13):
        working_df = download(i)
        working_df = new_format(working_df, dict_ids[i])

        if i > 0:
            working_df=working_df.drop(working_df.columns[1],axis=1)
        #only called once
        if final_df is None:
            final_df = working_df
        else:
            final_df = pd.merge(final_df, working_df, on=working_df.columns[0], how='inner', copy=True)
    
    final_df.to_csv(output_path, sep=',', index=False)

    #use gzip to compress .csv outputfile to <file_out>.gz
    path_object = Path(output_path)
    output_pathgz = path_object.with_suffix('.gz')
    final_df.to_csv(output_pathgz, sep=',', index=False, compression='gzip')




def download(download_id):
    #14 different files
    match download_id:
        # AUTOMATIC FREQUENCY RESTORATION
        case 0:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[18004368,18004369,18004370,18004351,18004371,18004372,18004373,18004374],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        # BALANCING ENERGY
        case 1:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[15004383,15004384,15004382,15004390],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')        
        # COSTS  
        case 2:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[16004391,16000419,16000418],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        # EXPORTED BALANCING SERVICES
        case 3:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[20004385],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        #FREQUENCY CONTAINMENT RESERVE
        case 4:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[17004363, 17004367],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        # IMPORTED BALANCING SERVICES
        case 5:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[21004386],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        # MANUAL FREQUENCY RESTORATION RESERVE
        case 6:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[19004377,19004375,19004376,19004352,19004378,19004379,19004380,19004381],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        
        #electricity consumption, actual
        case 7:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[5000410,5004387,5004359],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        #forecast consumption
        case 8:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[6000411,6004362],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        #electricity generation actual
        case 9:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[1001224,1004066,1004067,1004068,1001223,1004069,1004071,1004070,1001226,1001228,1001227,1001225],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        #electricity generation forecast
        case 10:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[2000122,2005097,2000715,2003791,2000123,2000125],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        #MARKET
        # CROSSBORDER FLOWS
        case 11:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[31004963,31004736,31004737,31004740,31004741,31004988,31004990,31004992,31004994,31004738,31004742,31004743,31004744,31004880,31004881,31004882,31004883,31004884,31004885,31004886,31004887,31004888,31004739],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        # CROSSBORDER SCHEDULED FLOWS
        case 12:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[22004629,22004722,22004724,22004404,22004409,22004545,22004546,22004548,22004550,22004551,22004552,22004405,22004547,22004403,22004406,22004407,22004408,22004410,22004412,22004549,22004553,22004998,22004712],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        # DAYAHEAD
        case 13:
            response = requests.post('https://www.smard.de/nip-download-manager/nip/download/market-data',
                                data='{"request_form":[{"format":"CSV","moduleIds":[8004169,8004170,8000251,8005078,8000252,8000253,8000254,8000255,8000256,8000257,8000258,8000259,8000260,8000261,8000262,8004996,8004997],"region":"DE","timestamp_from":1420066800000,"timestamp_to":'+str(int(datetime.datetime.today().timestamp()))+'000,"type":"discrete","language":"en","resolution":"hour"}]}')
        
    csvfile_data = response.content.decode('utf-8-sig')
    download_df = pd.read_csv(StringIO(csvfile_data), sep=";", header=[0], na_values='-', low_memory=False)
    
    return download_df



def new_format(df, my_dict):
        
    #use fitting dict to rename table head
    df.rename(columns=my_dict, inplace=True)
    
    #change Datetime_format; replace '-' with np.nan
    df['Start_Date'] = pd.to_datetime(df['Start_Date'])
    df['End_Date'] = pd.to_datetime(df['End_Date'])
    df.replace("-",np.nan, inplace=True)

    #remove , seperator for thousand
    df.replace(",","", inplace=True, regex=True)
    
    return df
    

def merge(fin_df, work_df, i):

    if i > 0:
        work_df=work_df.drop(work_df.columns[1],axis=1)
    
    fin_df = pd.merge(fin_df, work_df, on=work_df.columns[0], how='inner', copy=True)
    

output_path = '../data_collection/smard.csv'

dict_ids = [balancing_id["automatic_frequency"],
            balancing_id["balancing_energy"],
            balancing_id["costs"],
            balancing_id["exported_balancing_services"],
            balancing_id["frequency_containment"],
            balancing_id["imported_balancing_services"],
            balancing_id["manual_frequency"],
            electricity_consumption_id["actual"],
            electricity_consumption_id["forecast"],
            electricity_generation_id["actual"],
            electricity_generation_id["forecast"],
            market_id["cross_border_physical"],
            market_id["scheudled_commercial_exchanges"],
            market_id["day_ahead_prices"]    
    ]
    
final_df = None

for i in range(13):
    working_df = download(i)
    working_df = new_format(working_df, dict_ids[i])

    if i > 0:
       working_df=working_df.drop(working_df.columns[1],axis=1)
        #only called once
    if final_df is None:
            final_df = working_df
    else:
        final_df = pd.merge(final_df, working_df, on=working_df.columns[0], how='inner', copy=True)
    
final_df.to_csv(output_path, sep=',', index=False)

#use gzip to compress .csv outputfile to <file_out>.gz
path_object = Path(output_path)
output_pathgz = path_object.with_suffix('.gz')
final_df.to_csv(output_pathgz, sep=',', index=False, compression='gzip')

/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/3820852020.py:350: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Start_Date'] = pd.to_datetime(df['Start_Date'])
/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/3820852020.py:350: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Start_Date'] = pd.to_datetime(df['Start_Date'])
/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/3820852020.py:350: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Start_Date'] = pd.to_datetime(df['Start_Date'])
/var/folders/km/2p9w9k150f7ddl_dd2

This is responsible for the weather.

In [18]:
#Inititalisierung von Variablen und Ordnern
import time
import pandas as pd
import os
import concurrent.futures
import requests
import json
from datetime import datetime
import zipfile
import io
import re
import concurrent.futures
#Definiere abzufragende Stationen
combine_historicforecast_bool =False
station_ids_r = [ "01262", "01975", "02667"]
station_ids_f = [ "10870", "10147", "10513"]
station_place = [ "Muenchen", "Hamburg", "KoelnBonn" ]
#Ordnerstruktur für die Brechnung und Ausgabe
output_folder = "./weather/"
station_folder = "./weather/stations"
computing_folder = "./weather/computing_folder"
stations_combined = "./weather/stations_combined"
data_collection_folder="../data_collection"
forecas_folder="../forecast"
#Basis-URL für die DWD Wetterdaten
base_url_review = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/"
url_forecast = "https://dwd.api.proxy.bund.dev/v30/stationOverviewExtended"
# Nicht benötigte Spalten   
columns_remove_clouds = ["STATIONS_ID","eor", "QN_8","V_N_I"]
columns_remove_pressure = ["STATIONS_ID","eor", "QN_8"]
columns_remove_sun = ["STATIONS_ID","eor", "QN_7"]
columns_remove_temp = ["STATIONS_ID","QN_9", "eor"]
columns_remove_wind = ["STATIONS_ID","eor", "QN_3"]
columns_remove_precipitation = ["STATIONS_ID","eor", "QN_8", "WRTR", "RS_IND"]

columns_remove_forecast = ['isDay','dewPoint2m']
#URL Endungen für die Vergangenheit
data_types = {
    "temperature_historical": "air_temperature/historical/",
    "temperature_recent": "air_temperature/recent/",
    "cloudiness_historical": "cloudiness/historical/",
    "cloudiness_recent": "cloudiness/recent/",
    "pressure_historical": "pressure/historical/",
    "pressure_recent": "pressure/recent/",
    "sun_historical": "sun/historical/",
    "sun_recent": "sun/recent/",
    "wind_historical": "wind/historical/",
    "wind_recent": "wind/recent/",
    "precipitation_recent": "precipitation/recent/",
    "precipitation_historical": "precipitation/historical/",
}
#header für Api zugriff 
headers = {
    "accept": "application/json"
}
#Definitionen der Funktionen vom Basiswetterskript
def combine_historic(station_r, place): 
  #Kombiniere die Dateien paarweise
  try:
    file_r = os.path.join(station_folder, station_r, f"{station_r}_data_combined.csv")
    
    #Daten einlesen
    df_r = pd.read_csv(file_r)
    combined_df=df_r
    #Ausgabe-Dateiname
    output_file = os.path.join(stations_combined, f"{place}_review.csv")
    combined_df.to_csv(output_file, index=False)

    print(f"Kombiniert: {station_r} -> {output_file}")

  except FileNotFoundError as e:
    print(f"Datei nicht gefunden: {e}")
  except Exception as e:
    print(f"Fehler beim Verarbeiten von {station_r}: {e}")
def combine_all_stations():
  files = [f for f in os.listdir(stations_combined) if f.endswith('.csv')]

  #Umbennenen der Spalten nach Stationsnamen
  for file in files:
    file_path = os.path.join(stations_combined, file)
    df = pd.read_csv(file_path)
    #Extrahiere den Dateinamen
    file_name = os.path.splitext(file)[0]
    columname=[df.columns[0]] + [f'{col}_{file_name}' for col in df.columns[1:]]
    df.columns = columname
    print(f'Spalten umbennant für {file_name}')
    #station_column_filename = os.path.join(stations_combined, file_name)
    df.to_csv(file_path, index=False)

  #Verbinde alle DataFrames nebeneinander  
  all_data_frames = []
  for file in files:
    file_path = os.path.join(stations_combined, file)  
    
    #Lade Daten aus Datei und füge sie zur Liste
    try:
      df = pd.read_csv(file_path, delimiter=",", parse_dates=["date"], date_format="%Y%m%d%H")
      all_data_frames.append(df)
      print(f"Daten hinzugefügt von: {file_path}")
    except Exception as e:
      print(f"Fehler beim Laden der Datei {file}: {e}")
  
  #Wenn geladen wurden -> kombiniere
  if all_data_frames:
    combined_data = all_data_frames[0]
    for df in all_data_frames[1:]:
      #Test MESS_DATUM als Datum
      df["date"] = pd.to_datetime(df["date"], errors="coerce")                
      #Daten zusammenführen
      combined_data = pd.merge(combined_data, df, on=[  "date"], how="outer")

    #Sortieren und doppelte löschen
    combined_data = combined_data.sort_values(by=[  "date"]).drop_duplicates(subset=[  "date"], keep='last')

  #Speichern
  final_filename = os.path.join(data_collection_folder, f"weather.csv")
  combined_data.to_csv(final_filename, index=False)
  print(f"Alle kombinierten Daten gespeichert in: {final_filename}")
def start_combine_historic():
    max_workers = min(os.cpu_count(), len(station_ids_r))  #Maximal so viele Stationen wie vorhanden oder CPU Anzahl
    print(f"Starte die Verknüfung aller Daten für {len(station_ids_r)} Stationen mit {max_workers} Threads.")
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        #Jede Station wird parallel heruntergeladen
        future_to_station = {executor.submit(combine_historic, station_r, place): (station_r,  place) for station_r,  place in zip(station_ids_r,  station_place) }    
        for future in concurrent.futures.as_completed(future_to_station):
            station_id = future_to_station[future]
            try:
                future.result()  #Funktion ausführen und Fehler abfangen
                print(f"Dateien verknüpft aller Daten für Station {station_id}.")
            except Exception as e:
                print(f"Fehler beim Verknüfung aller Daten für Station {station_id}: {e}")
def combine_forecast():

  files = [f for f in os.listdir(station_folder) if f.endswith('.csv')]

  #Umbennenen der Spalten nach Stationsnamen
  for file in files:
    file_path = os.path.join(station_folder, file)
    df = pd.read_csv(file_path)
    #Extrahiere den Dateinamen
    file_name = os.path.splitext(file)[0]
    columname=[df.columns[0]] + [f'{col}_{file_name}' for col in df.columns[1:]]
    df.columns = columname
    print(f'Spalten umbennant für {file_name}')
    #station_column_filename = os.path.join(stations_combined, file_name)
    df.to_csv(file_path, index=False)

  #Verbinde alle DataFrames nebeneinander  
  all_data_frames = []
  for file in files:
    file_path = os.path.join(station_folder, file)  
    
    #Lade Daten aus Datei und füge sie zur Liste
    try:
      df = pd.read_csv(file_path, delimiter=",", parse_dates=["date"], date_format="%Y%m%d%H")
      all_data_frames.append(df)
      print(f"Daten hinzugefügt von: {file_path}")
    except Exception as e:
      print(f"Fehler beim Laden der Datei {file}: {e}")
  
  #Wenn geladen wurden -> kombiniere
  if all_data_frames:
    combined_data = all_data_frames[0]
    for df in all_data_frames[1:]:
      #Test MESS_DATUM als Datum
      df["date"] = pd.to_datetime(df["date"], errors="coerce")                
      #Daten zusammenführen
      combined_data = pd.merge(combined_data, df, on=[  "date"], how="outer")

    #Sortieren und doppelte löschen
    combined_data = combined_data.sort_values(by=[  "date"]).drop_duplicates(subset=[  "date"], keep='last')


  final_filename = os.path.join(forecas_folder, f"weather_forecast.csv")
  combined_data.to_csv(final_filename, index=False)
  print(f"Kombinierter Forecast gespeichert in: {final_filename}")
def create_folder():
  os.makedirs(computing_folder, exist_ok=True)
  os.makedirs(stations_combined, exist_ok=True)
  for station in station_ids_r:
    output_folder_station = os.path.join(computing_folder, station)
    os.makedirs(output_folder_station, exist_ok=True)
    station_folder =os.path.join(output_folder,'stations',station)
    os.makedirs(station_folder, exist_ok=True)

#Wetter Reviewfunktionen:
#Funktion zur Suche und Herunterladen der Wetterdaten pro Station
def station_folderget_weather_data_for_station_review(station_id):
    #os.makedirsrs(computing_folder, exist_ok=True)
    #os.makedirsrs(station_folder, exist_ok=True)
    output_filepath = os.path.join(computing_folder,station_id)
    #os.makedirsrs(output_filepath, exist_ok=True)
    print(f"Speicherort {output_filepath}, computing_folder {computing_folder}, station_id {station_id}")    
    for data_type, endpoint in data_types.items():
        url = base_url_review + endpoint
        
        #Esrtellt Liste von Dateien im Verzeichnis
        response = requests.get(url)
        response.raise_for_status()

        #Suche nach passender ZIP-Datei
        for line in response.text.splitlines():
            if station_id in line and "zip" in line:
                filename = re.search(r'href="(.*?)"', line).group(1)
                file_url = url + filename
                
                #Lade ZIP-Datei herunter
                print(f"Lade herunter: {file_url}")
                file_response = requests.get(file_url)
                file_response.raise_for_status()
                
                #Entpacke ZIP-Datei und suche passender TXT-Datei in der ZIP
                with zipfile.ZipFile(io.BytesIO(file_response.content)) as z:
                    if data_type == "cloudiness_historical" or data_type == "cloudiness_recent":
                        txt_files = [name for name in z.namelist() if re.match(r'produkt_n_stunde_\d{8}_\d{8}_' + station_id + r'\.txt', name)]
                    elif data_type == "pressure_historical" or data_type == "pressure_recent":
                        txt_files = [name for name in z.namelist() if re.match(r'produkt_p0_stunde_\d{8}_\d{8}_' + station_id + r'\.txt', name)]
                    elif data_type == "sun_historical" or data_type == "sun_recent":
                        txt_files = [name for name in z.namelist() if re.match(r'produkt_sd_stunde_\d{8}_\d{8}_' + station_id + r'\.txt', name)]
                    elif data_type == "wind_historical" or data_type == "wind_recent":
                        txt_files = [name for name in z.namelist() if re.match(r'produkt_ff_stunde_\d{8}_\d{8}_' + station_id + r'\.txt', name)]
                    elif data_type == "precipitation_historical" or data_type == "precipitation_recent":
                        txt_files = [name for name in z.namelist() if re.match(r'produkt_rr_stunde_\d{8}_\d{8}_' + station_id + r'\.txt', name)]
                    else:
                        txt_files = [name for name in z.namelist() if re.match(r'produkt_tu_stunde_\d{8}_\d{8}_' + station_id + r'\.txt', name)]
                    
                    if not txt_files:
                        print(f"Keine TXT-Datei im erwarteten Format für Station {station_id} gefunden.")
                        continue  

                    #Wenn TXT-Datei gefunden wurde, lade sie in pandas
                    txt_filename = txt_files[0]
                    with z.open(txt_filename) as f:
                        #Test ob ladbar
                        try:
                            df = pd.read_csv(f, sep=";", encoding="utf-8")
                            if df.empty:
                                print(f"Warnung: Die Datei {txt_filename} ist leer.")
                            else:
                                print("Daten geladen für:", txt_filename)

                                #Ausgabeordener checken

                                #Dateinamen nach Datenart setzen
                                if data_type == "temperature_historical":
                                    new_filename = f"temp_{station_id}_hist.txt"
                                elif data_type == "temperature_recent":
                                    new_filename = f"temp_{station_id}_recent.txt"
                                elif data_type == "cloudiness_historical":
                                    new_filename = f"clouds_{station_id}_hist.txt"
                                elif data_type == "cloudiness_recent":
                                    new_filename = f"clouds_{station_id}_recent.txt"
                                elif data_type == "pressure_historical":
                                    new_filename = f"pressure_{station_id}_hist.txt"
                                elif data_type == "pressure_recent":
                                    new_filename = f"pressure_{station_id}_recent.txt"
                                elif data_type == "sun_historical":
                                    new_filename = f"sun_{station_id}_hist.txt"
                                elif data_type == "sun_recent":
                                    new_filename = f"sun_{station_id}_recent.txt"
                                elif data_type == "wind_historical":
                                    new_filename = f"wind_{station_id}_hist.txt"
                                elif data_type == "wind_recent":
                                    new_filename = f"wind_{station_id}_recent.txt"      
                                elif data_type == "precipitation_historical":
                                    new_filename = f"precipitation_{station_id}_hist.txt"
                                elif data_type == "precipitation_recent":
                                    new_filename = f"precipitation_{station_id}_recent.txt"                                
                                
                                #Speichere TXT-Datei im angegebenen Ordner
                                #print(f"Speicherort {output_filepath}, computing_folder {computing_folder}, station_id {station_id}, new_filename {new_filename}")
                                output_filename = os.path.join(output_filepath, new_filename)                                
                                df.to_csv(output_filename, sep=";", encoding="utf-8", index=False)
                                print(f"Wetterdaten gespeichert unter: {output_filepath}")   
                                print(f"Die Datei wurde erfolgreich gespeichert unter: {os.path.abspath(output_filepath)}")
                        except Exception as e:
                            print(f"Fehler beim Laden der Datei {txt_filename}: {e}")
    cut_historic_bevor_2015(station_id)
#Funktion zum Herunterladen der Wetterdaten für alle angegebenen Stationen
def download_weather_data_for_all_stations_review(station_ids):
    max_workers = min(os.cpu_count(), len(station_ids))  #Maximal 10 Threads oder so viele Stationen wie vorhanden
    print(f"Starte den Download für {len(station_ids)} Stationen mit {max_workers} Threads.")
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        #Jede Station wird parallel heruntergeladen
        future_to_station = {executor.submit(station_folderget_weather_data_for_station_review, station_id): station_id for station_id in station_ids}        
        for future in concurrent.futures.as_completed(future_to_station):
            station_id = future_to_station[future]
            try:
                future.result()  #Funktion ausführen und Fehler abfangen
                print(f"Download abgeschlossen für Station {station_id}.")
            except Exception as e:
                print(f"Fehler beim Herunterladen von Daten für Station {station_id}: {e}")
def cut_historic_bevor_2015(station_id):
    computing_folder_station = os.path.join(computing_folder, station_id)
    station_files = [f for f in os.listdir(computing_folder_station) if re.match(r'(.+)_hist\.txt', f)]    
    for file in station_files:
        file_path = os.path.join(computing_folder_station, file)
        with open(file_path, 'r') as infile:
            lines = infile.readlines()
        
        #Filtert Zeilen nach 2015 sind
        filtered_lines = []
        for line in lines[:1]:
            filtered_lines.append(line)
        for line in lines[1:]:
            columns = line.strip().split(';')
            if len(columns) > 1:  
                mess_datum = columns[1]
                year = int(mess_datum[:4])                
                if year >= 2015:
                    filtered_lines.append(line)

        #Schreibe Zeilen in die Datei zurück
        with open(file_path, 'w') as outfile:
            outfile.writelines(filtered_lines)
        print(f"Historisch bis 2015 gekürzt: {file}")
    
    #Aufruf nur benutzen, wenn start_... in weather nicht ausgeführt wird
    remove_columns_review(station_id)
def start_cut_historic_bevor_2015(station_ids):
    max_workers = min(os.cpu_count(), len(station_ids))  #Maximal 10 Threads oder so viele Stationen wie vorhanden
    print(f"Starte die Kürzung bis 2015 für {len(station_ids)} Stationen mit {max_workers} Threads.")
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        #Jede Station wird parallel heruntergeladen
        future_to_station = {executor.submit(cut_historic_bevor_2015, station_id): station_id for station_id in station_ids}        
        for future in concurrent.futures.as_completed(future_to_station):
            station_id = future_to_station[future]
            try:
                future.result()  #Funktion ausführen und Fehler abfangen
                print(f"Dateien bis 2015 gekürzt für Station {station_id}.")
            except Exception as e:
                print(f"Fehler beim Kürzen bis 2015 für Station {station_id}: {e}")
def remove_columns_review(station_id):
    print('Start Remove Columns')
    computing_folder_station =os.path.join(computing_folder, station_id)
    temp_files = [f for f in os.listdir(computing_folder_station) if f.startswith("temp_") and f.endswith(".txt")]
    clouds_files = [f for f in os.listdir(computing_folder_station) if f.startswith("clouds_") and f.endswith(".txt")]
    pressure_files = [f for f in os.listdir(computing_folder_station) if f.startswith("pressure_") and f.endswith(".txt")]
    sun_files = [f for f in os.listdir(computing_folder_station) if f.startswith("sun_") and f.endswith(".txt")]
    wind_files = [f for f in os.listdir(computing_folder_station) if f.startswith("wind_") and f.endswith(".txt")]
    precipitation_files = [f for f in os.listdir(computing_folder_station) if f.startswith("precipitation_") and f.endswith(".txt")]
    
    for file in clouds_files:
        file_path = os.path.join(computing_folder_station, file)        
        try:
            df = pd.read_csv(file_path, delimiter=";", skipinitialspace=True)            
            #Entferne die Spalten, ganz oben definiert
            df = df.drop(columns=[col for col in columns_remove_clouds if col in df.columns])            
            #Speichere modifizierte Datei
            df.to_csv(file_path, sep=";", index=False)
            print(f"Spalten aus {file} entfernt.")
        
        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file}: {e}")
    
    for file in pressure_files:
        file_path = os.path.join(computing_folder_station, file)
        
        try:
            df = pd.read_csv(file_path, delimiter=";", skipinitialspace=True)            
            #Entferne die Spalten, ganz oben definiert
            df = df.drop(columns=[col for col in columns_remove_pressure if col in df.columns])            
            #Speichere modifizierte Datei
            df.to_csv(file_path, sep=";", index=False)
            print(f"Spalten aus {file} entfernt.")
        
        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file}: {e}")

    for file in sun_files:
        file_path = os.path.join(computing_folder_station, file)
        
        try:
            df = pd.read_csv(file_path, delimiter=";", skipinitialspace=True)            
            #Entferne die Spalten, ganz oben definiert
            df = df.drop(columns=[col for col in columns_remove_sun if col in df.columns])            
            #Speichere modifizierte Datei
            df.to_csv(file_path, sep=";", index=False)
            print(f"Spalten aus {file} entfernt.")
        
        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file}: {e}")

    for file in temp_files:
        file_path = os.path.join(computing_folder_station, file)
        
        try:
            df = pd.read_csv(file_path, delimiter=";", skipinitialspace=True)            
            #Entferne die Spalten, ganz oben definiert
            df = df.drop(columns=[col for col in columns_remove_temp if col in df.columns])            
            #Speichere modifizierte Datei
            df.to_csv(file_path, sep=";", index=False)
            print(f"Spalten aus {file} entfernt.")
        
        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file}: {e}")

    for file in wind_files:
        file_path = os.path.join(computing_folder_station, file)
        
        try:
            df = pd.read_csv(file_path, delimiter=";", skipinitialspace=True)            
            #Entferne die Spalten, ganz oben definiert
            df = df.drop(columns=[col for col in columns_remove_wind if col in df.columns])            
            #Speichere modifizierte Datei
            df.to_csv(file_path, sep=";", index=False)
            print(f"Spalten aus {file} entfernt.")
        
        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file}: {e}")

    for file in precipitation_files:
        file_path = os.path.join(computing_folder_station, file)
        
        try:
            df = pd.read_csv(file_path, delimiter=";", skipinitialspace=True)            
            #Entferne die Spalten, ganz oben definiert
            df = df.drop(columns=[col for col in columns_remove_precipitation if col in df.columns])            
            #Speichere modifizierte Datei
            df.to_csv(file_path, sep=";", index=False)
            print(f"Spalten aus {file} entfernt.")
        
        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file}: {e}")

    #Aufruf nur benutzen, wenn start_... in weather nicht ausgeführt wird
    combine_historic_recent(station_id)
def start_remove_columns_review(station_ids):
    max_workers = min(os.cpu_count(), len(station_ids))  #Maximal 10 Threads oder so viele Stationen wie vorhanden
    print(f"Starte die Löschen von Spalten für {len(station_ids)} Stationen mit {max_workers} Threads.")
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        #Jede Station wird parallel heruntergeladen
        future_to_station = {executor.submit(remove_columns_review, station_id): station_id for station_id in station_ids}        
        for future in concurrent.futures.as_completed(future_to_station):
            station_id = future_to_station[future]
            try:
                future.result()  #Funktion ausführen und Fehler abfangen
                print(f"Spalten gelöscht für Station {station_id}.")
            except Exception as e:
                print(f"Fehler beim Löschen von Spalten für Station {station_id}: {e}")
def combine_historic_recent(station_id):
    computing_folder_station = os.path.join(computing_folder, station_id)    
    #Suche nach Dateien für jeweilige Station
    station_files = [f for f in os.listdir(computing_folder_station) if re.match(r'(.+)_' + station_id + r'_(hist|recent)\.txt', f)]
    
    #Gruppiere Dateien nach Wettertyp und Station-ID
    file_pairs = {}
    for file in station_files:
        match = re.match(r'(.+)_' + station_id + r'_(hist|recent)\.txt', file)
        if match:
            wettertyp, period = match.groups()  #Wettertyp und Zeitraum
            key = f"{wettertyp}_{station_id}"
            if key not in file_pairs:
                file_pairs[key] = {}
            file_pairs[key][period] = os.path.join(computing_folder_station, file)

    #Führe historische und aktuelle Daten zusammen
    for key, file_pair in file_pairs.items():
        if 'hist' in file_pair and 'recent' in file_pair:
            #Einlesen historische, aktuellen Daten
            hist_df = pd.read_csv(file_pair['hist'], delimiter=";")
            recent_df = pd.read_csv(file_pair['recent'], delimiter=";")
            hist_df["MESS_DATUM"] = pd.to_datetime(hist_df["MESS_DATUM"], format="%Y%m%d%H", errors="coerce")
            recent_df["MESS_DATUM"] = pd.to_datetime(recent_df["MESS_DATUM"], format="%Y%m%d%H", errors="coerce")

            #Kombinieren Daten und entferne Duplikaten
            combined_df = pd.concat([hist_df, recent_df]).drop_duplicates(subset=["MESS_DATUM"], keep='last')
            combined_df = combined_df.sort_values(by=["MESS_DATUM"])
            combined_df["MESS_DATUM"] = combined_df["MESS_DATUM"].dt.strftime("%Y%m%d%H")

            #Speichern unter kombinierten Namen
            combined_filename = os.path.join(computing_folder_station, f"{key}_combined.txt")
            combined_df.to_csv(combined_filename, sep=";", index=False)
            print(f"Kombinierte Datei gespeichert: {combined_filename}")
        else:
            print(f"Fehlende Datei für {key}: entweder historische oder aktuelle Datei fehlt.")
    
    #Aufruf nur benutzen, wenn start_... in weather nicht ausgeführt wird
    combine_all_station_data_review(station_id)
def start_combine_historic_recent(station_ids):
    max_workers = min(os.cpu_count(), len(station_ids))  #Maximal 10 Threads oder so viele Stationen wie vorhanden
    print(f"Starte die Verknüfung Historsich mit Aktuell für {len(station_ids)} Stationen mit {max_workers} Threads.")
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        #Jede Station wird parallel heruntergeladen
        future_to_station = {executor.submit(combine_historic_recent, station_id): station_id for station_id in station_ids}        
        for future in concurrent.futures.as_completed(future_to_station):
            station_id = future_to_station[future]
            try:
                future.result()  #Funktion ausführen und Fehler abfangen
                print(f"Dateien verknüpft Historsich mit Aktuell für Station {station_id}.")
            except Exception as e:
                print(f"Fehler beim Verknüfung Historsich mit Aktuell für Station {station_id}: {e}")
def combine_all_station_data_review(station_id):
    computing_folder_station = os.path.join(computing_folder, station_id)
    station_folder_station = os.path.join(station_folder, station_id) 
    #os.makedirsrs(station_folder_station, exist_ok=True)  
    #Suche nach Dateien mit dem Suffix "_combined" 
    combined_files = [f for f in os.listdir(computing_folder_station) if f.endswith(f"_{station_id}_combined.txt")]
    all_data_frames = []
    #print(f"Combined Files: {combined_files}")
    for file in combined_files:
        file_path = os.path.join(computing_folder_station, file)
        
        #Lade Daten aus Datei und füge sie zur Liste
        try:
            df = pd.read_csv(file_path, delimiter=";", parse_dates=["MESS_DATUM"], date_format="%Y%m%d%H")
            all_data_frames.append(df)
            print(f"Daten hinzugefügt von: {file_path}")
        except Exception as e:
            print(f"Fehler beim Laden der Datei {file}: {e}")

    #print("Alle Dateien geladen")
    #Wenn geladen wurden -> kombiniere
    if all_data_frames:
        combined_data = all_data_frames[0]
        for df in all_data_frames[1:]:
            #Test MESS_DATUM als Datum
            df["MESS_DATUM"] = pd.to_datetime(df["MESS_DATUM"], format="%Y%m%d%H", errors="coerce")                
            #Daten zusammenführen
            combined_data = pd.merge(combined_data, df, on=[  "MESS_DATUM"], how="outer")

        #Sortieren und doppelte löschen
        combined_data = combined_data.sort_values(by=[  "MESS_DATUM"]).drop_duplicates(subset=[  "MESS_DATUM"], keep='last')
        combined_data["MESS_DATUM"] = combined_data["MESS_DATUM"].dt.strftime("%Y%m%d%H")
        
        # Header ändern
        header_mapping = {
            "STATIONS_ID": "STATIONS_ID",
            "MESS_DATUM": "date",
            "V_N_I": "Wolken_Interp",
            "V_N": "clouds",
            "P": "stationPressure_hPa",
            "P0": "surfacePressure_hPa",
            "SD_SO": "sunshine_min",
            "TT_TU": "T_temperature_C",
            "RF_TU": "humidity_Percent",
            "F": "wind_speed_ms",
            "D": "wind_direction_degree",
            "R1": "precipitationTotal_mm",
            "RS_IND": "precipitation_indicator"

        }
    
        combined_data.rename(columns=header_mapping, inplace=True)

        #Speichern in Datei
        final_filename = os.path.join(station_folder_station, f"{station_id}_data_combined.csv")
        combined_data.to_csv(final_filename, sep=",", index=False)
        print(f"Alle kombinierten Daten für Station {station_id} gespeichert in: {final_filename}")

    else:
        print(f"Keine kombinierten Dateien für Station {station_id} gefunden.")
def start_combine_all_station_data_review(station_ids):
    max_workers = min(os.cpu_count(), len(station_ids))  #Maximal 10 Threads oder so viele Stationen wie vorhanden
    print(f"Starte die Verknüfung aller Daten für {len(station_ids)} Stationen mit {max_workers} Threads.")
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        #Jede Station wird parallel heruntergeladen
        future_to_station = {executor.submit(combine_all_station_data_review, station_id): station_id for station_id in station_ids}        
        for future in concurrent.futures.as_completed(future_to_station):
            station_id = future_to_station[future]
            try:
                future.result()  #Funktion ausführen und Fehler abfangen
                print(f"Dateien verknüpft aller Daten für Station {station_id}.")
            except Exception as e:
                print(f"Fehler beim Verknüfung aller Daten für Station {station_id}: {e}")
#Wetter Forecastfunktionen:
def get_weather_data_for_station_forecast(station_id, station_place):
    params = {
        "stationIds": station_id
    }
    #Anfrage vorbereiten
    request = requests.Request("GET", url_forecast, headers=headers, params=params)
    prepared_request = request.prepare()
    
    response = requests.Session().send(prepared_request)
    #Ausgabeordener checken
    #os.makedirs(computing_folder, exist_ok=True)
    #os.makedirs(station_folder, exist_ok=True)
    if response.status_code == 200:
        data = response.json()
        
        filename = os.path.join(computing_folder, f"weather_forecast_{station_place}.json")
        with open(filename, "w") as file:
            json.dump(data, file, indent=4)
        print(f"Die Wettervorhersage wurde in {filename} gespeichert.")
        
        #JSON-Daten laden und verarbeiten
        with open(filename) as file:
            data = json.load(file)
        
        for station_id, station_data in data.items():
            forecast_data = station_data["forecast1"]
            start_time = forecast_data["start"]
            time_step = forecast_data["timeStep"]

            date = [datetime.utcfromtimestamp((start_time + i * time_step) / 1000) for i in range(len(forecast_data["temperature"]))]
            
            variables = {
                "T_temperature_C": forecast_data.get("temperature", []),
                "T_temperature_standarddeviation_C": forecast_data.get("temperatureStd", []),
                "precipitationTotal_mm": forecast_data.get("precipitationTotal", []),
                "sunshine_min": forecast_data.get("sunshine", []),
                "dewPoint2m": forecast_data.get("dewPoint2m", []),
                "surfacePressure_hPa": forecast_data.get("surfacePressure", []),
                "humidity_Percent": forecast_data.get("humidity", []),
                "isDay_bool": forecast_data.get("isDay", []),
                #"icon": forecast_data.get("icon", []),
                #"icon1h": forecast_data.get("icon1h", [])
            }
            
            #Alle Listen auf gleiche Länge bringen
            max_length = max(len(date), *(len(values) for values in variables.values()))
            date.extend([None] * (max_length - len(date)))  # date auf max. Länge auffüllen
            for key, values in variables.items():
                variables[key].extend([None] * (max_length - len(values)))  # Werte-Listen auffüllen
            
            # DataFrame erstellen
            df = pd.DataFrame({
                "date": date,
                **variables
            })

            #DataFrame Temperatur von ZehntelGrad in Grad             
            df["T_temperature_C"] = df["T_temperature_C"].apply(lambda x: x / 10 if pd.notnull(x) else x)
            df["T_temperature_standarddeviation_C"] = df["T_temperature_standarddeviation_C"].apply(lambda x: x / 10 if pd.notnull(x) else x)
            df["surfacePressure_hPa"] = df["surfacePressure_hPa"].apply(lambda x: x / 10 if pd.notnull(x) else x)
            df["humidity_Percent"] = df["humidity_Percent"].apply(lambda x: x / 10 if pd.notnull(x) else x)

            #Date ins richte Foramt konvertieren
            df["date"] = df["date"].apply(lambda x: x.strftime("%Y%m%d%H"))

            df.to_csv(os.path.join(station_folder, f"weather_forecast_{station_place}.csv"), index=False)
            print(f"Die Wettervorhersage wurde in weather_forecast_{station_place}.csv konvertiert")
    else:
        print(f"Fehler bei der Anfrage: {response.status_code}")
def download_weatherforecast_data_for_all_stations_forecast(station_ids, station_places):
    for (station_id , station_place) in zip(station_ids, station_places):
        print(f"Starte den Download für Station {station_id}...")
        get_weather_data_for_station_forecast(station_id, station_place)
        print()
def remove_columns_forecast():
    print("Starte den Spaltenentfernumg")
    forecast_files = [f for f in os.listdir(station_folder) if f.startswith("weather_forecast_")]  
    print(f"File: {forecast_files}...")
    for file in forecast_files:
        print(f"Starte den Spaltenentfernumg für {file}...")
        file_path = os.path.join(station_folder, file)
        
        try:
            df = pd.read_csv(file_path, delimiter=",", skipinitialspace=True)  
            print(f"Spalten im DataFrame: {list(df.columns)}")          
            #Entferne die Spalten, ganz oben definiert
            df = df.drop(columns=[col for col in columns_remove_forecast if col in df.columns])     
            #Speichere modifizierte Datei
            df.to_csv(file_path, sep=",", index=False)
            print(f"Spalten aus {file} entfernt.")
        
        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file}: {e}")
#Starte Download und Verarbeitung der Wetterdaten
start = time.time()
#Erstelle die Ordner
create_folder()

#Starte Rückblick-Download
download_weather_data_for_all_stations_review(station_ids_r)

end = time.time()
verstrichene_zeit = end - start
print(f'Ausführungszeit: {verstrichene_zeit} Sekunden')

#Starte Vorhersagen-Download
download_weatherforecast_data_for_all_stations_forecast(station_ids_f, station_place)
remove_columns_forecast()

end = time.time()
verstrichene_zeit = end - start
print(f'Ausführungszeit: {verstrichene_zeit} Sekunden')

#Kombiniere die historischen und vorhergesagten Daten
start_combine_historic()
end = time.time()
verstrichene_zeit = end - start
print(f'Ausführungszeit: {verstrichene_zeit} Sekunden')
#Alle Stationen kombinieren
combine_all_stations()
combine_forecast()

end = time.time()
verstrichene_zeit = end - start
print(f'Ausführungszeit: {verstrichene_zeit} Sekunden')

Starte den Download für 3 Stationen mit 3 Threads.
Speicherort ./weather/computing_folder/01262, computing_folder ./weather/computing_folder, station_id 01262
Speicherort ./weather/computing_folder/01975, computing_folder ./weather/computing_folder, station_id 01975
Speicherort ./weather/computing_folder/02667, computing_folder ./weather/computing_folder, station_id 02667
Lade herunter: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/air_temperature/historical/stundenwerte_TU_01975_19490101_20231231_hist.zipLade herunter: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/air_temperature/historical/stundenwerte_TU_02667_19600101_20231231_hist.zip

Lade herunter: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/air_temperature/historical/stundenwerte_TU_01262_19920517_20231231_hist.zip
Daten geladen für: produkt_tu_stunde_19920517_20231231_01262.txt
Wetterdaten gespeichert unter: ./we

/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/2155369691.py:596: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = [datetime.utcfromtimestamp((start_time + i * time_step) / 1000) for i in range(len(forecast_data["temperature"]))]
/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/2155369691.py:596: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = [datetime.utcfromtimestamp((start_time + i * time_step) / 1000) for i in range(len(forecast_data["temperature"]))]
/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/2155369691.py:596: DeprecationWarning: datetime.dat

Die Wettervorhersage wurde in ./weather/computing_folder/weather_forecast_Hamburg.json gespeichert.
Die Wettervorhersage wurde in weather_forecast_Hamburg.csv konvertiert

Starte den Download für Station 10513...
Die Wettervorhersage wurde in ./weather/computing_folder/weather_forecast_KoelnBonn.json gespeichert.
Die Wettervorhersage wurde in weather_forecast_KoelnBonn.csv konvertiert

Starte den Spaltenentfernumg
File: ['weather_forecast_Muenchen.csv', 'weather_forecast_KoelnBonn.csv', 'weather_forecast_Hamburg.csv']...
Starte den Spaltenentfernumg für weather_forecast_Muenchen.csv...
Spalten im DataFrame: ['date', 'T_temperature_C', 'T_temperature_standarddeviation_C', 'precipitationTotal_mm', 'sunshine_min', 'dewPoint2m', 'surfacePressure_hPa', 'humidity_Percent', 'isDay_bool']
Spalten aus weather_forecast_Muenchen.csv entfernt.
Starte den Spaltenentfernumg für weather_forecast_KoelnBonn.csv...
Spalten im DataFrame: ['date', 'T_temperature_C', 'T_temperature_standarddeviation_C', 'p

In [21]:
df_res = pd.read_csv('../data_collection/merged_data.csv')
df_ens = pd.read_csv('../data_collection/merged_data3.csv')
df_smard = pd.read_csv('../data_collection/smard.csv')
df_smard = df_smard.rename(columns={'Start_Date': 'Date'})
df.to_csv('../data_collection/smard.csv', index=False)
df_smard = pd.read_csv('../data_collection/smard.csv')
df_smard['Date'] = pd.to_datetime(df['Date'])
df_filtered = df[df['Date'].dt.minute == 0]
df_filtered['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_filtered.to_csv('../data_collection/smard.csv', index=False)
df_smard = pd.read_csv('../data_collection/smard.csv')

df_weather = pd.read_csv('../data_collection/weather.csv')
df_weather = df_weather.rename(columns={'date': 'Date'})
df.to_csv('../data_collection/weather.csv', index=False)
df_weather = pd.read_csv('../data_collection/weather.csv')
df_weather['Date'] = pd.to_datetime(df['Date'])
df_filtered = df[df['Date'].dt.minute == 0]
df_filtered['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_filtered.to_csv('../data_collection/weather.csv', index=False)
df_weather = pd.read_csv('../data_collection/weather.csv')
df_covid = pd.read_csv('../data_collection/covid.csv')

df_social = pd.read_csv('../data_collection/major_social_events.csv')
df_carbon = pd.read_csv('../data_collection/carbon_price_forward_filled.csv')

merge_big = pd.merge(df_ens, df_res, on='Date', how='outer')
merge_big = pd.merge(merge_big, df_smard, on='Date', how='outer')
merge_big = pd.merge(merge_big, df_social, on='Date', how='outer')
merge_big = pd.merge(merge_big, df_carbon, on='Date', how='outer')
merge_big = pd.merge(merge_big, df_weather, on='Date', how='outer')
merge_big = pd.merge(merge_big, df_covid, on='Date', how='outer')
merge_big.to_csv('../allData.csv', index=False)

print("CSV files have been merged and saved.")

/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/513469211.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d %H:%M:%S')
/var/folders/km/2p9w9k150f7ddl_dd2zttlg00000gn/T/ipykernel_14891/513469211.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d %H:%M:%S')


CSV files have been merged and saved.
